## Preparing data for Word2Vec and exporting as a dataset to use in Word2vec

Using this approach: https://muhark.github.io/python/ml/nlp/2021/10/21/word2vec-from-scratch.html

In [124]:
import sys
!{sys.executable} -m pip  install --quiet torch duckdb duckdb-engine ipython-sql sqlalchemy python-snappy pyarrow memray pandas watermark ipywidgets bs4 polars rich humanize matplotlib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
%load_ext sql

In [5]:
import pandas as pd
import json

# display entire row
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [6]:
# DuckDB Magics
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [7]:
# Connect to DuckDB
%sql duckdb:///my-db.duckdb

In [ ]:
# Create parquet file 
# Use bdt tool https://github.com/andygrove/bdt/blob/main/src/parquet.rs
# bdt convert goodreads_books.json goodreads_parquet.parquet

In [490]:
# Create from parquet file
%sql CREATE TABLE books_pqt AS SELECT * FROM '/Users/vicki/viberary/viberary/notebooks/goodreads_parquet.parquet/*.parquet';

,Count
0,2360655


In [17]:
%sql describe books_pqt

,column_name,column_type,null,key,default,extra
0,asin,VARCHAR,YES,None,None,None
1,authors,"STRUCT(author_id VARCHAR, role VARCHAR)[]",YES,None,None,None
2,average_rating,VARCHAR,YES,None,None,None
3,book_id,VARCHAR,YES,None,None,None
4,country_code,VARCHAR,YES,None,None,None
5,description,VARCHAR,YES,None,None,None
6,edition_information,VARCHAR,YES,None,None,None
7,format,VARCHAR,YES,None,None,None
8,image_url,VARCHAR,YES,None,None,None
9,is_ebook,VARCHAR,YES,None,None,None


In [65]:
# Inspect data
%sql  text_data << select title, description, popular_shelves from books_pqt;

Returning data to local variable text_data


In [66]:
text_data['popular_shelves'] = text_data['popular_shelves'].replace(regex="\'",value="\"")

In [ ]:
exploded = (pd.concat({i: pd.DataFrame(x) for i, x in text_data.pop('popular_shelves').items()})
         .reset_index(level=1, drop=True)
         .join(text_data)
         .reset_index(drop=True))

In [59]:
# Description field to string
exploded["description_string"] = exploded.description.astype(str)
exploded["description_string"] = exploded.description_string.replace('None','',regex=True)

In [61]:
# Count of popular shelves tags to int
exploded['count'] = exploded['count'].astype(int)
exploded['weighted_count'] = exploded['count']* (exploded['name'] + " ")

In [62]:
# Let's start with a simple model using title and description and tags
exploded['word2vec_data'] = (exploded['title']+ " ") + (exploded['description_string']+ " ") + exploded['weighted_count']

In [63]:
# Include weighted tags in word2vec input
word2vec = exploded.groupby(["title","description"])["weighted_count"].apply(','.join).reset_index()
word2vec['word2vec_data'] = (word2vec['description']+ " ") + (word2vec['title']+ " ") + word2vec['weighted_count']
word2vec

title  \
0                All's Fairy in Love and War (Avalon: Web of Magic, #8)   
1                                                  Best Friends Forever   
2                                                           Good Harbor   
3                                     Playmaker: A Venom Series Novella   
4  Runic Astrology: Starcraft and Timekeeping in the Northern Tradition   
5                                         The Aeneid for Boys and Girls   
6                                                  The Devil's Notebook   
7                   The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)   
8                                                 The Wanting of Levine   
9                                           W.C. Fields: A Life on Film   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              description  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                           To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war. Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: to put an end to the war using her diplomatic skills.\nAll's Fairy In Love And War is the eighth book in Avalon: Web of Magic, a twelve-book fantasy series for middle grade readers. Through their magical journey, the teenage heroines discover who they really are . . . and run into plenty of good guys, bad guys, and cute guys. Out of print for two years, Seven Seas is pleased to return the Avalon series to print in editions targeted for today's readers, with new manga-style covers and interior illustrations.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [64]:
# Preprocessing of data for word2vec

# Lower case
word2vec['word2vec_data'].str.lower()

# Remove everything except a-z@

# Split on spaces

# Remove stopwords

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [125]:
import torch